# FSM: 有限状态机

## 是什么？

### 简单来说

- 有限状态机称有限状态自动机，简称状态机，是种算法思想
- 是表示有限个状态以及在这些状态之间的转移和动作等行为的数学模型。
- 一般有限状态机由一组状态、一个初始状态、输入和根据输入及现有状态转换为下一个状态的转换函数组成

### 逻辑

![](https://upload.wikimedia.org/wikipedia/commons/thumb/6/64/Finite_State_Machine_Logic.svg/400px-Finite_State_Machine_Logic.svg.png)

### 数学模型

有限状态机是五元组 $(\Sigma ,S,s_{0},\delta ,F)$，这里的：

- $\Sigma$ 是输入字母表（符号的非空有限集合）。
- $S$ 是状态的非空有限集合。
- $s_{0} \in S$ 是初始状态。
- $\delta$ 是状态转移函数：$\delta :S\times \Sigma \rightarrow S$。
- $F \subseteq S$ 是接受状态集

### 例子

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9d/DFAexample.svg/440px-DFAexample.svg.png)  

- $\Sigma: \{0, 1\}$
- $S: \{S_1, S_2\}$
- $S_{1} \in S$ 是初始状态
- $F = \{S_1\}$
- $\delta$: (S1, 0) -> S2, (S1, 1) -> S1, (S2, 0) -> S1 and (S2, 1) -> S2


   | 0 | 1
-- |---|---
S1 | S2| S1
S2 | S1| S2

## 怎么做？

- 一系列有限度的，有明确顺序的状态
- 状态间的切换，也是明确的不变的条件
- 这种情况，使用 FSM 来解决就对了  

举个例子：  

网页上有一个菜单元素。  
- 鼠标悬停的时候，菜单显示；
- 鼠标移开的时候，菜单隐藏。  

如果使用有限状态机描述，就是这个菜单只有两种状态（显示和隐藏），鼠标会引发状态转变。

### Python 代码

We want to recognize the meaning of very small sentences with an extremely limited vocabulary and syntax:  
These sentences should start with "Python is" followed by
- an adjective or
- the word "not" followed by an adjective.

e.g.  
"Python is great" → positive meaning  
"Python is stupid" → negative meaning  
"Python is not ugly" → positive meaning  

![](http://www.python-course.eu/images/finite_state_machine_example.png)

In [3]:
# Python FSM Demo
class StateMachine:
    def __init__(self):
        self.handlers = {}
        self.startState = None
        self.endStates = []
    # 添加一个状态
    def add_state(self, name, handler, end_state=0):
        name = name.upper()
        # handler 是定义的迁移条件和目标状态函数
        self.handlers[name] = handler
        print(handler)
        if end_state:
            print('ha')
            self.endStates.append(name)
    # 设置起始状态
    def set_start(self, name):
        self.startState = name.upper()

    def run(self, cargo):
        try:
            handler = self.handlers[self.startState]
        except:
            raise InitializationError("must call .set_start() before .run()")
        if not self.endStates:
            raise  InitializationError("at least one state must be an end_state")
    
        while True:
            (newState, cargo) = handler(cargo)
            # endStates 即手动添加的 pos，neg 和 error
            if newState.upper() in self.endStates:
                print("reached ", newState)
                break 
            else:
                handler = self.handlers[newState.upper()]


positive_adjectives = ["great","super", "fun", "entertaining", "easy"]
negative_adjectives = ["boring", "difficult", "ugly", "bad"]

# 每个单词迁移条件及目标状态

# start state
def start_transitions(txt):
    # txt.split(None,1) 1 为分开后的最高序列 ID
    # 'Python is a' split 后为 'Python', 'is a'
    splitted_txt = txt.split(None,1)
    word, txt = splitted_txt if len(splitted_txt) > 1 else (txt,"")
    if word == "Python":
        newState = "Python_state"
    else:
        newState = "error_state"
    return (newState, txt)

# is state
def python_state_transitions(txt):
    splitted_txt = txt.split(None,1)
    word, txt = splitted_txt if len(splitted_txt) > 1 else (txt,"")
    if word == "is":
        newState = "is_state"
    else:
        newState = "error_state"
    return (newState, txt)

# next state is 'is'
def is_state_transitions(txt):
    splitted_txt = txt.split(None,1)
    word, txt = splitted_txt if len(splitted_txt) > 1 else (txt,"")
    # not state
    if word == "not":
        newState = "not_state"
    elif word in positive_adjectives:
        newState = "pos_state"
    elif word in negative_adjectives:
        newState = "neg_state"
    else:
        newState = "error_state"
    return (newState, txt)

# next state is 'not'
# 'not' state is a state of transition
def not_state_transitions(txt):
    splitted_txt = txt.split(None,1)
    word, txt = splitted_txt if len(splitted_txt) > 1 else (txt,"")
    if word in positive_adjectives:
        newState = "neg_state"
    elif word in negative_adjectives:
        newState = "pos_state"
    else:
        newState = "error_state"
    return (newState, txt)

def neg_state(txt):
    print("Hallo")
    return ("neg_state", "")

if __name__== "__main__":
    m = StateMachine()
    # 添加状态
    m.add_state("Start", start_transitions)
    m.add_state("Python_state", python_state_transitions)
    m.add_state("is_state", is_state_transitions)
    m.add_state("not_state", not_state_transitions)
    # 接受（结束）状态
    m.add_state("neg_state", None, end_state=1)
    m.add_state("pos_state", None, end_state=1)
    m.add_state("error_state", None, end_state=1)
    # 设置初始状态
    m.set_start("Start")
    # 运行实例
    m.run("Python is great")
    m.run("Python is difficult")
    m.run("Perl is ugly")
    m.run("Python is not difficult")
    m.run("Python isn't great")

<function start_transitions at 0x10459fd08>
<function python_state_transitions at 0x10459fb70>
<function is_state_transitions at 0x10459fae8>
<function not_state_transitions at 0x10459fa60>
None
ha
None
ha
None
ha
reached  pos_state
reached  neg_state
reached  error_state
reached  pos_state
reached  error_state


In [330]:
# Python transitions Demo
from transitions import Machine
import random

class PythonState(object):
    states = ['pos_state', 'neg_state', 'error_state', 'Python_state', 'is_state', 'not_state', 'Start_state', 
              'pos_neg_not_state', 'pos_neg_state']
    positive_adjectives = ["great","super", "fun", "entertaining", "easy"]
    negative_adjectives = ["boring", "difficult", "ugly", "bad"]

    def __init__(self, txt):

        self.splitted_txt = txt.split()
        self.words = self.splitted_txt if len(self.splitted_txt) > 1 else (txt)
        
        # Initialize the state machine
        self.machine = Machine(model=self, states=PythonState.states, initial='Start_state')
        
        self.machine.add_transition(trigger='is_Python', source='Start_state', dest='Python_state', 
                                    conditions=['is_python'])
        self.machine.add_transition(trigger='is_Python', source='Start_state', dest='error_state')  
                
        self.machine.add_transition('is_Is', 'Python_state', 'is_state',
                                   conditions=['is_is'])
        self.machine.add_transition('is_Is', 'Python_state', 'error_state')

        
        self.machine.add_transition('is_Pos_Neg_Not', 'is_state', 'pos_neg_not_state',
                                   conditions=['is_pos_neg_not'])
        self.machine.add_transition('is_Pos_Neg_Not', 'is_state', 'error_state')

        
        self.machine.add_transition('is_Pos_Neg', 'pos_neg_not_state', 'pos_neg_state',
                                   conditions=['is_pos_neg'])
        self.machine.add_transition('is_Pos_Neg', 'pos_neg_not_state', 'not_state')


        self.machine.add_transition('is_Pos', 'pos_neg_state', 'pos_state',
                                   conditions=['is_pos'])
        self.machine.add_transition('is_Pos', 'pos_neg_state', 'neg_state')

        
        self.machine.add_transition('is_Pos', 'not_state', 'neg_state',
                                   conditions=['is_not_pos'])
        self.machine.add_transition('is_Pos', 'not_state', 'pos_state')

    def is_python(self):
        return self.words[0] == 'Python'
    
    def is_is(self):
        return self.words[1] == 'is'
    
    def is_pos_neg_not(self):
        return self.words[2] == 'not' or self.words[2] in (positive_adjectives + negative_adjectives)
    
    def is_pos_neg(self):
        return self.words[2] != 'not' and self.words[2] in (positive_adjectives + negative_adjectives)

    def is_pos(self):
        return self.words[2] in positive_adjectives
    
    def is_not_pos(self):
        return self.words[3] in positive_adjectives

In [331]:
def test(text):
    text = PythonState(text)
    states = []
    try:
        text.is_Python()
        states.append(text.state)
        text.is_Is()
        states.append(text.state)
        text.is_Pos_Neg_Not()
        states.append(text.state)
        text.is_Pos_Neg()
        states.append(text.state)
        text.is_Pos()
        states.append(text.state)
    except:
        pass
    return states[-1]

In [332]:
print(test("Python is great"))
print(test("Python is difficult"))
print(test("Perl is ugly"))
print(test("Python is not difficult"))
print(test("Python isn't great"))

pos_state
neg_state
error_state
pos_state
error_state


In [337]:
# 另一段 Demo
from transitions import Machine
import random

class NarcolepticSuperhero(object):

    # Define some states. Most of the time, narcoleptic superheroes are just like
    # everyone else. Except for...
    states = ['asleep', 'hanging out', 'hungry', 'sweaty', 'saving the world']

    def __init__(self, name):

        # No anonymous superheroes on my watch! Every narcoleptic superhero gets
        # a name. Any name at all. SleepyMan. SlumberGirl. You get the idea.
        self.name = name

        # What have we accomplished today?
        self.kittens_rescued = 0

        # Initialize the state machine
        self.machine = Machine(model=self, states=NarcolepticSuperhero.states, initial='asleep')

        # Add some transitions. We could also define these using a static list of
        # dictionaries, as we did with states above, and then pass the list to
        # the Machine initializer as the transitions= argument.

        # At some point, every superhero must rise and shine.
        self.machine.add_transition(trigger='wake_up', source='asleep', dest='hanging out')

        # Superheroes need to keep in shape.
        self.machine.add_transition('work_out', 'hanging out', 'hungry')

        # Those calories won't replenish themselves!
        self.machine.add_transition('eat', 'hungry', 'hanging out')

        # Superheroes are always on call. ALWAYS. But they're not always
        # dressed in work-appropriate clothing.
        self.machine.add_transition('distress_call', '*', 'saving the world',
                         before='change_into_super_secret_costume')

        # When they get off work, they're all sweaty and disgusting. But before
        # they do anything else, they have to meticulously log their latest
        # escapades. Because the legal department says so.
        self.machine.add_transition('complete_mission', 'saving the world', 'sweaty',
                         after='update_journal')

        # Sweat is a disorder that can be remedied with water.
        # Unless you've had a particularly long day, in which case... bed time!
        self.machine.add_transition('clean_up', 'sweaty', 'asleep', conditions=['is_exhausted']) # 各 50% 可能性
        self.machine.add_transition('clean_up', 'sweaty', 'hanging out')

        # Our NarcolepticSuperhero can fall asleep at pretty much any time.
        self.machine.add_transition('nap', '*', 'asleep')

    def update_journal(self):
        """ Dear Diary, today I saved Mr. Whiskers. Again. """
        self.kittens_rescued += 1

    def is_exhausted(self):
        """ Basically a coin toss. """
        return random.random() < 0.5

    def change_into_super_secret_costume(self):
        print("Beauty, eh?")

In [341]:
batman = NarcolepticSuperhero("Batman")
batman.state

'asleep'

In [342]:
batman.wake_up()
batman.work_out()
batman.state

'hungry'

In [343]:
batman.kittens_rescued

0

In [344]:
batman.distress_call()

Beauty, eh?


True

In [345]:
batman.state

'saving the world'

In [346]:
batman.complete_mission()
batman.state

'sweaty'

In [347]:
batman.clean_up()
batman.state

'hanging out'

In [348]:
batman.kittens_rescued

1

### JS 代码  

交通信号灯（红绿灯）    
[JavaScript与有限状态机 - 阮一峰的网络日志](http://www.ruanyifeng.com/blog/2013/09/finite-state_machine_for_javascript.html)  
[jakesgordon/javascript-state-machine: A javascript finite state machine library](https://github.com/jakesgordon/javascript-state-machine)

>一个对象的状态越多、发生的事件越多，就越适合采用有限状态机的写法。  
另外，JavaScript 语言是一种异步操作特别多的语言，常用的解决方法是指定回调函数，但这样会造成代码结构混乱、难以测试和除错等问题。  
有限状态机提供了更好的办法：把异步操作与对象的状态改变挂钩，当异步操作结束的时候，发生相应的状态改变，由此再触发其他操作。  
这要比回调函数、事件监听、发布/订阅等解决方案，在逻辑上更合理，更易于降低代码的复杂度。

In [ ]:
　　var fsm = StateMachine.create({
　　
　　　　initial: 'green',
　　
　　　　events: [
　　　　　　{ name: 'warn',  from: 'green',  to: 'yellow' },
　　　　　　{ name: 'stop', from: 'yellow', to: 'red' },
　　　　　　{ name: 'ready',  from: 'red',    to: 'yellow' },
　　　　　　{ name: 'go', from: 'yellow', to: 'green' }
　　　　]
　　
　　});

生成实例以后，就可以随时查询当前状态。  
* `fsm.current` ：返回当前状态。
* `fsm.is(s)` ：返回一个布尔值，表示状态 s 是否为当前状态。
* `fsm.can(e)` ：返回一个布尔值，表示事件 e 是否能在当前状态触发。
* `fsm.cannot(e)` ：返回一个布尔值，表示事件 e 是否不能在当前状态触发。

JFSM 允许为每个**事件**或**状态**指定两个回调函数：  

* `onbeforewarn`：在warn事件发生之前触发。
* `onafterwarn`（可简写成 onwarn） ：在 warn 事件发生之后触发。
* `onwarn`：在 warn 时间发生之间。
* `onleavegreen` ：在离开 green 状态时触发。
* `onentergreen`（可简写成 ongreen） ：在进入 green 状态时触发。  

假定 warn 事件使得状态从 green 变为 yellow，上面四类回调函数的发生顺序如下：  
onbeforewarn → onleavegreen → onenteryellow → onafterwarn。

除了为每个事件和状态单独指定回调函数，还可以为所有的事件和状态指定通用的回调函数。  

* `onbeforeevent` ：任一事件发生之前触发。
* `onleavestate` ：离开任一状态时触发。
* `onevent`: 在事件发生期间触发。
* `onenterstate` ：进入任一状态时触发。
* `onafterevent` ：任一事件结束后触发。

如果事件的回调函数里面有异步操作（比如与服务器进行 Ajax 通信），这时我们可能希望等到异步操作结束，再发生状态改变。  
这就要用到 transition 方法。

In [ ]:
　　fsm.onleavegreen = function(){
        # 异步操作 light.fadeOut
　　　　light.fadeOut('slow', function() {
　　　　　　fsm.transition();
　　　　});
        # 如果不希望状态立即改变，就要让回调函数返回 StateMachine.ASYNC，表示状态暂时不改变；
        # 等到异步操作结束，再调用 transition 方法，使得状态发生改变。
　　　　return StateMachine.ASYNC;
　　};

In [ ]:
# 测试代码
var fsm = new StateMachine({
    init: 'solid',
    transitions: [
      { name: 'melt',     from: 'solid',  to: 'liquid' },
      { name: 'freeze',   from: 'liquid', to: 'solid'  },
      { name: 'vaporize', from: 'liquid', to: 'gas'    },
      { name: 'condense', from: 'gas',    to: 'liquid' }
    ],
    methods: {
      # 事件发生后触发，等于 onAfter
      onAfterMelt:  function()       { console.log('I melted')    },
      # 事件发生前触发
      onBeforeMelt: function()       { console.log('I am frozing')},
      # 进入状态时触发  等于 onentergas
      onEnterGas:   function()       { console.log('I am liquid, want to be gas') },
      # 离开状态时触发
      onLeaveGas:   function()       { console.log('I have just condensed from gas to liquid') }
    }
  });

In [ ]:
# test
var StateMachine = require('javascript-state-machine');
var fsm = new StateMachine({
    init: 'solid',
    transitions: [
      { name: 'melt',     from: 'solid',  to: 'liquid' },
      { name: 'freeze',   from: 'liquid', to: 'solid'  },
      { name: 'vaporize', from: 'liquid', to: 'gas'    },
      { name: 'condense', from: 'gas',    to: 'liquid' }
    ],
    methods: {
      onBeforeMelt:  function()       { console.log('I am melting')},
      onAfterMelt:   function()       { console.log('I melted')    },
      onEnterSolid:  function()       { console.log('I am solid, want to be liquid') },
      onLeaveLiquid: function()       { console.log('I have just frozen from liquid to solid') }
    }
  });

运行结果：  

```
I am solid, want to be liquid
undefined
> fsm.melt()
I am melting
I melted
true
> fsm.state
'liquid'
> fsm.freeze()
I have just frozen from liquid to solid
I am solid, want to be liquid
true
>```

### 常见应用  


搜索引擎分词，词法分析器，正则表达式引擎，xml/json/css解析器， 游戏 NPC 设计等




[有限状态机](http://helloitworks.com/754.html)

## 为什么？

如果使用状态机：  
只要描定义出所有状态，明白状态之间的转换，然后执行相应的动作  

一大堆的 if elif else 判定树就可以全部丢掉了! 变成类似的非常简单的:  
**将复杂的有前后顺序要求的流程上下文判定，都丢给有限状态机去处理。**

## 小结

- 一种基于图（节点为状态，边为转移路径）的模式指定方式，一种抽象
- 在 FSM 眼里，程序就是从一种状态转移到另一种状态，通过确定从起始状态到某个用相应序列标记的接受状态之间是否存在路径（可能性）

## 参考文献及资料


- [确定和不确定的有限自动机，以及正则表达式——形式语言和自动机的那一套理论【1】 - 知乎专栏](https://zhuanlan.zhihu.com/p/27149604)
- [第 10 章　模式、自动机和正则表达式-图灵社区](http://www.ituring.com.cn/book/tupubarticle/5512)
- Python FSM 库
    - [pytransitions/transitions: A lightweight, object-oriented finite state machine implementation in Python](https://github.com/pytransitions/transitions)
    - [mriehl/fysom: Finite State Machine for Python (based on Jake Gordon's javascript-state-machine)](https://github.com/mriehl/fysom)
- JS FSM 库
  - [jakesgordon/javascript-state-machine: A javascript finite state machine library](https://github.com/jakesgordon/javascript-state-machine)
- [可爱的 Python：使用状态机](https://www.ibm.com/developerworks/cn/linux/sdk/python/python-4/index.html)
- [chaos2wechat/fsm.rst at master · ZoomQuiet/chaos2wechat](https://github.com/ZoomQuiet/chaos2wechat/blob/master/source/ch03/fsm.rst#pyfsm)
- [JavaScript与有限状态机 - 阮一峰的网络日志](http://www.ruanyifeng.com/blog/2013/09/finite-state_machine_for_javascript.html)
- C 实现：[有限状态机](http://helloitworks.com/754.html)
- C 设计：[有限状态机（FSM）的设计与实现（一） - 平凡之路 - 博客园](http://www.cnblogs.com/chencheng/archive/2012/06/25/2562660.html)
- Java 实现：[有限状态机FSM的几种简单实现 - 简书](http://www.jianshu.com/p/3ca1ff2d7344)
- Go 实现：[一个有特色的有限状态机 | 鸟窝](http://colobu.com/2016/12/24/a-featured-fsm/)  
- Python 实现：[[python]有限状态机（FSM）简单实现 - 赖勇浩的编程私伙局 - CSDN博客](http://blog.csdn.net/gzlaiyonghao/article/details/1510688)
- Erlang OTP：[Gen_Fsm行为 — Erlang OTP Design Principles v0.1 documentation](http://erlang.shiningray.cn/otp-design-principles/gen_fsm.html)
- [自述 | Redux 中文文档 Join the chat at https://gitter.im/camsong/redux-in-chinese](http://cn.redux.js.org/)
- Coursera 课程：[自动机理论 Automata](http://mooc.guokr.com/course/418/Automata/)